# Импортируем модель

In [1]:
from tensorflow import keras

print('Load model')

model = keras.models.load_model('models/sgsol_2021')

2023-05-10 13:43:13.354701: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 13:43:13.438402: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 13:43:13.441130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 13:43:14.763347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Load model


# Импортируем базу данных формата TDB

In [2]:
from pycalphad import Database
from tqdm import tqdm
import modules.variables as var

print('load TDB file')
# constants:
P = 101325
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)
all_components = var.all_components
all_phases = var.all_phases

load TDB file


/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `K` was defined in the following phases: ['FCC_A1', 'FCC_L12'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `S` was defined in the following phases: ['FCC_A1'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typechar}` was defined in the following phases: "
/home/dmitry/.local/lib/python3.10/site-packages/pycalphad/io/tdb.py:925: UserWarning: The type definition character `P` was defined in the following phases: ['FCC_A1', 'FCC_4SL', 'HCP_A3', 'HCP_4SL'], but no corresponding TYPE_DEFINITION line was found in the TDB.
  warnings.warn(f"The type definition character `{typec

# Задаем условия для компонент [CO, CR, MO, NI, TA, V, W, ZR]	

In [3]:
amounts = [[0.10640018, 0.07053918, 0.09098721, 0.08310691, 0.1866029,
            0.18583018, 0.17365684, 0.1028766],
           [0.05581862, 0.06488962, 0.21049123, 0.10578128, 0.08068205,
            0.13075854, 0.22992805, 0.12165063],
           [0.0770591, 0.05441745, 0.11515643, 0.17607291, 0.2857176,
            0.12056114, 0.0763749, 0.09464048]]
temp = [1000] * len(amounts)
components = ['CO', 'CR', 'MO', 'NI', 'TA', 'V', 'W', 'ZR']	

In [4]:
from modules.GetGm import getGM
from modules.filters import getPossiblePhases

possible_phases = getPossiblePhases(db, components)

# Расчет энергии Гиббса

In [5]:
print("Расчет энергий Гиббса")
gms = []
for i in tqdm(range(len(amounts))):
    gms.append(getGM(db, components, all_phases, possible_phases, amounts[i], temp[i]))

Расчет энергий Гиббса


100%|██████████| 3/3 [00:48<00:00, 16.27s/it]


# Обработка энергий Гиббса

In [6]:
import pandas as pd
from modules.filters import normGms, normTemp

min_temp = 298.15
max_temp = 3000

print("Обработка энергий Гиббса")

# Нормализаиция энергий Гиббса
X = normGms(gms, all_phases)

# Добавление значение температуры в данные для расчета
if type(temp) == int or type(temp) ==  float:
    X = X.copy()
    X.loc[:, 'Temp'] = temp
else:
    X = pd.concat([X, pd.DataFrame(temp, columns=['Temp'])], axis=1)
    # X = pd.concat([X, pd.DataFrame(temp.to_list(), columns=['Temp'])], axis=1)
# Нормализация значения температуры
X = normTemp(X, min_temp, max_temp)

# Добавление количество компонентов в данные для расчета
X_A = pd.DataFrame(amounts, columns=components)
# X_A = pd.DataFrame(amounts.to_list(), columns=components)
X = pd.concat([X, X_A], axis=1)

X

Обработка энергий Гиббса


,A15_CR3PT,A2_BCC,AG2BA,AG2BA3,AG2CA,AG2CD3,AG2CE,AG2DY,AG2ER,AG2GA,...,ZRSI2,Temp,CO,CR,MO,NI,TA,V,W,ZR
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.23395,0.106400,0.070539,0.090987,0.083107,0.186603,0.185830,0.173657,0.102877
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.23395,0.055819,0.064890,0.210491,0.105781,0.080682,0.130759,0.229928,0.121651
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.23395,0.077059,0.054417,0.115156,0.176073,0.285718,0.120561,0.076375,0.094640


# Предсказание стабильных фаз

In [7]:
print("Обработка энергий Гиббса")

results = model.predict(X)

Обработка энергий Гиббса
1/1 [==============================] - 0s 165ms/step


# Фильтрация результатов

In [8]:
from modules.filters import parsePrediction

filter_value = 0.2

print("Фильтрация энергий Гиббса")

for i in range(len(results)):
    print(parsePrediction(results[i], all_phases, filter_value))

Фильтрация энергий Гиббса
{'MU_PHASE': 0.24300249, 'LAVES_C15': 0.25761005, 'LAVES_C14': 0.48049244}
{'MU_PHASE': 0.22020783, 'LAVES_C15': 0.2622267, 'LAVES_C14': 0.4980362}
{'LAVES_C15': 0.25295833, 'MU_PHASE': 0.2545565, 'LAVES_C14': 0.4744896}


In [9]:
from modules.filters import parsePredictionKeys

print(f"Значение фильтра = {filter_value}")
for i in range(len(results)):
    print(f"Концентрация = {amounts[i]}\nТемпература = {temp[i]}\n",
          f"Фазы: {parsePredictionKeys(results[i], all_phases, filter_value)}\n")

Значение фильтра = 0.2
Концентрация = [0.10640018, 0.07053918, 0.09098721, 0.08310691, 0.1866029, 0.18583018, 0.17365684, 0.1028766]
Температура = 1000
 Фазы: ['MU_PHASE', 'LAVES_C15', 'LAVES_C14']

Концентрация = [0.05581862, 0.06488962, 0.21049123, 0.10578128, 0.08068205, 0.13075854, 0.22992805, 0.12165063]
Температура = 1000
 Фазы: ['MU_PHASE', 'LAVES_C15', 'LAVES_C14']

Концентрация = [0.0770591, 0.05441745, 0.11515643, 0.17607291, 0.2857176, 0.12056114, 0.0763749, 0.09464048]
Температура = 1000
 Фазы: ['LAVES_C15', 'MU_PHASE', 'LAVES_C14']

